## <center>Circular tree with cubic Bezier edges</center>

This notebook illustrates how a tree can be read from a GML-format file, and how its vertex coordinates, returned by the Buchheim layout from the Julia package, `NetworkLayout.jl`, are mapped on circles centered at the root. The edges of the circular tree are plotted as cubic Bezier  curves defined as paths, via PlotlyJS.jl.

As example we use the tree, having  Professor Gilbert Strang (GS) as root. The tree structure  was retrieved from the Mathematics Genealogy Project [https://www.mathgenealogy.org/id.php?id=13383](https://www.mathgenealogy.org/id.php?id=13383).
The  GS profile was scrapped in 2019, using the Python tool geneagrapher (ggrapher) [https://github.com/davidalber/geneagrapher](https://github.com/davidalber/geneagrapher).
After   data wrangling  a GML file was setup, using python igraph library. At the time of scrapping the tree contained 195 nodes (vertices). Now Mathematics Genealogy Project displays more than 200  descendants for GS.

The GML file is read  by `GraphIO.jl` as a `SimpleDiGraph{Int64}`, defined by `LightGraphs.jl`. Since `GraphIO.jl` cannot retrieve metadata from a GML file, we saved metadata associated to each vertex in a JSON file, and  here we are using that information as hovering data in the corresponding plotly plot of the tree.

We note that our graph has a leaf with two parents, i.e. a PhD recipient with two advisors. This slight deviation from  a tree definition  was not an impediment to define the SimpleDiGraph, by its list of lists representation.

In [1]:
using LightGraphs,  ParserCombinator, GraphIO, NetworkLayout
using PlotlyJS, JSON

function proj2circle(xpos::Vector{Float64}, ypos::Vector{Float64}, direction=-1,  θ₀=-π/2)
    # function to project to circles the vertex coordinates for the same level vertices in the Buchheim tree
    #  (xv, yv)  in zip(xpos, ypos) is a vertex position  assigned by Buchheim layout
    # direction: 1 or -1 points the counter clockwise, resp clockwise mapping of vertices on the circular layout
    # θ₀: starting θ for vertices
    xmin, xmax = extrema(xpos)
    ymin, ymax = extrema(ypos)
    θ = 2π*(xpos .- xmin)/(xmax-xmin) .+ θ₀
    r = ymax .-  ypos
    xc = r .* cos.(θ)
    yc = direction*r .* sin.(θ)
    return xc, yc
end    

function  Bezier_edges(xc::Vector{Float64}, yc::Vector{Float64},  edges::Vector{Tuple{Int64, Int64}})
    # xc, yc Vectors  of vertex coordinates on circles around the root
    # edges as vector of tuples, (i, j)
    shapes =  []
    for e in edges
        #set the coordinates of the Bezier control points b_i(xi, yi), for the cubic Bezier representing a tree edge
        x1 = xc[e[1]] 
        y1 = yc[e[1]]
        d1 = sqrt(x1^2+y1^2)
        x4 = xc[e[2]] 
        y4 = yc[e[2]] 
        d4 = sqrt(x4^2+y4^2)
        d = 0.5*(d1+d4)
        if d1 == 0 || d4 == 0 # the first Bezier  control point is  is the  root;  plot the edge as a segment of line
            x2 = x1
            y2 = y1
            x3 = x4
            y3 = 0.3*y1 + 0.7*y4
        else   
            x2 = d*x1/d1
            y2 = d*y1/d1
            x3 = d*x4/d4
            y3 = d*y4/d4
        end       
        push!(shapes, attr(type="path",
                           layer="below",
                           #Bezier path representing an edge
                           path= "M $x1 $y1, C $x2 $y2, $x3 $y3, $x4  $y4",
                           line=attr(color="rgb(210,210,210)", width=1)
                                ))
    end
    return shapes
end    

WebIO._IJuliaInit()

Bezier_edges (generic function with 1 method)

In [2]:
G = loadgraph("data/gstrang_descendants.gml",  "digraph", GraphIO.GML.GMLFormat())
E = [(e.src, e.dst) for e in edges(G)];
node_pos = buchheim(G.fadjlist)
xpos = Float64[]
ypos = Float64[]
for npos in node_pos
    push!(xpos, npos[1])
    push!(ypos, npos[2])
end

xc, yc = proj2circle(xpos, ypos)
N = length(xc)
vmetadata = String[]
open("data/gstrang_descinfo.json", "r") do f
    global vmetadata
    jdata = JSON.read(f, String)  
    vmetadata = JSON.parse(jdata)  
end

#define the plotly trace for circular graph vertices

circ_nodes = scatter(
                x=xc, 
                y=yc, 
                mode="markers",
                name=" ",
                marker=attr(size=5, #vcat([10], repeat([5], N-1)), 
                            color="rgb(255, 200, 0)",
                            line=attr(color="rgb(255, 200, 0)", width=0.5)),
                text=vmetadata,
                hoverinfo="text")


scatter with fields hoverinfo, marker, mode, name, text, type, x, and y


In [3]:
layout = Layout(
         title_text="Gilbert Strang's Descendants in the Mathematics Genealogy Project",
         title_x=0.5,
         width=725, 
         height=675,
         font_family="Balto",
         showlegend=false,
         xaxis_visible=false,
         yaxis_visible=false, 
         plot_bgcolor="rgb(10,10,10)")

layout["shapes"] = Bezier_edges(xc, yc, E)
layout["annotations"] = [attr(showarrow=false,
                                 text="Gilbert<br>Strang",
                                 xref="x",
                                 yref="y",
                                 x=xc[1]-0.05,
                                 y=yc[1]-0.1,           
                                 font=attr(family="Balto", size=14, color="white"),
                                 xanchor="center",   
                                 yanchor="top"
                                )];


pl = Plot(circ_nodes, layout)

data: [
  "scatter with fields hoverinfo, marker, mode, name, text, type, x, and y"
]

layout: "layout with fields annotations, font, height, margin, plot_bgcolor, shapes, showlegend, template, title, width, xaxis, and yaxis"